In [1]:
import os, sys

import numpy as np
import pandas as pd 
from datasets import load_dataset

import importlib
from tqdm import tqdm
from joblib import Parallel, delayed
from copy import copy

from transformers import (
    AutoConfig,
    AutoTokenizer,
    FlaxAutoModelForSequenceClassification,
    HfArgumentParser,
    PretrainedConfig,
    TrainingArguments,
    is_tensorboard_available,
)

data_root = "/kaggle/input/feedback-prize-effectiveness/"
train = pd.read_csv("/kaggle/input/feedback-prize-effectiveness/train.csv")


/kaggle/working/jax/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
LABEL_MAPPING = {"Ineffective": 0, "Adequate": 1, "Effective": 2}

def _prepare_training_data_helper(args, tokenizer, df, is_train):
    training_samples = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        idx = row["essay_id"]
        discourse_text = row["discourse_text"]
        discourse_type = row["discourse_type"]

        if is_train:
            filename = os.path.join(args.input, "train", idx + ".txt")
        else:
            filename = os.path.join(args.input, "test", idx + ".txt")

        with open(filename, "r") as f:
            text = f.read()

        encoded_text = tokenizer.encode_plus(
            discourse_type + " " + discourse_text,
            text,
            add_special_tokens=False,
        )
        input_ids = encoded_text["input_ids"]

        sample = {
            "discourse_id": row["discourse_id"],
            "input_ids": input_ids,
            # "discourse_text": discourse_text,
            # "essay_text": text,
            # "mask": encoded_text["attention_mask"],
        }

        if "token_type_ids" in encoded_text:
            sample["token_type_ids"] = encoded_text["token_type_ids"]

        label = row["discourse_effectiveness"]

        sample["label"] = LABEL_MAPPING[label]

        training_samples.append(sample)
    return training_samples


def prepare_training_data(df, tokenizer, args, num_jobs, is_train):
    training_samples = []

    df_splits = np.array_split(df, num_jobs)

    results = Parallel(n_jobs=num_jobs, backend="multiprocessing")(
        delayed(_prepare_training_data_helper)(args, tokenizer, df, is_train) for df in df_splits
    )
    for result in results:
        training_samples.extend(result)

    return training_samples

In [3]:
sys.path.append("../configs")
cfg = copy(importlib.import_module("default_config").cfg)

# Load pretrained model and tokenizer
config = AutoConfig.from_pretrained(
    cfg.model_name_or_path,
    num_labels=cfg.num_labels,
    #finetuning_task=data_args.task_name,
    #use_auth_token=True if cfg.use_auth_token else None,
)
tokenizer = AutoTokenizer.from_pretrained(
    cfg.model_name_or_path,
    use_fast=not cfg.use_slow_tokenizer,
    #use_auth_token=True if cfg.use_auth_token else None,
)
model = FlaxAutoModelForSequenceClassification.from_pretrained(
    cfg.model_name_or_path,
    config=config,
    #use_auth_token=True if cfg.use_auth_token else None,
)

normalizer.cc(51) LOG(INFO) precompiled_charsmap is empty. use identity normalization.
Downloading: 100%|██████████| 489M/489M [00:12<00:00, 41.1MB/s] 
Some weights of the model checkpoint at google/bigbird-roberta-base were not used when initializing FlaxBigBirdForSequenceClassification: {('cls', 'predictions', 'transform', 'LayerNorm', 'scale'), ('cls', 'predictions', 'transform', 'dense', 'kernel'), ('cls', 'predictions', 'bias'), ('cls', 'seq_relationship', 'kernel'), ('cls', 'seq_relationship', 'bias'), ('cls', 'predictions', 'transform', 'dense', 'bias'), ('cls', 'predictions', 'transform', 'LayerNorm', 'bias')}
- This IS expected if you are initializing FlaxBigBirdForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing FlaxBigBirdForSequenceClassification from the checkpoint of a model that

In [4]:
import json
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
## stratified Kfold for train dataframe using discourse_type and discourse_effectiveness
for fold, (train_index, valid_index) in enumerate(kf.split(train, train["discourse_type"], train["discourse_effectiveness"])):

    train_temp = train.iloc[train_index]
    valid_temp = train.iloc[valid_index]

    train_data = prepare_training_data(train_temp, tokenizer, cfg, num_jobs=32, is_train=True)
    val_data = prepare_training_data(valid_temp, tokenizer, cfg, num_jobs=32, is_train=True)

    df = pd.DataFrame.from_records(train_data)
    df.to_json(f"/kaggle/working/folds/train_{fold}.jsonl", orient="records", lines=True)

    df = pd.DataFrame.from_records(val_data)
    df.to_json(f"/kaggle/working/folds/valid_{fold}.jsonl", orient="records", lines=True)

    print("Fold:", fold)
    print("Train:", train_index)
    print("Valid:", valid_index)
    print("\n")

100%|██████████| 229/229 [00:00<00:00, 587.08it/s]
Token indices sequence length is longer than the specified maximum sequence length for this model (9275 > 4096). Running this sequence through the model will result in indexing errors
100%|██████████| 229/229 [00:00<00:00, 444.65it/s]
https://symbolize.stripped_domain/r/?trace=7f262c81f60e,7f26ffd7f0bf,fffffffffffffffe&map=c6154ce86c34fa08dd761dfb5fd1bf97e63ed5c3:7f262c588000-7f262ce098f8 
*** SIGTERM received by PID 94751 (TID 94751) on cpu 59 from PID 88996; stack trace: ***
PC: @     0x7f262c81f60e  (unknown)  _$LT$hashbrown..raw..RawTable$LT$T$C$A$GT$$u20$as$u20$core..ops..drop..Drop$GT$::drop::h8aeca1e9ba6ed7af
    @     0x7f261dfb7353        992  (unknown)
    @     0x7f26ffd7f0c0  (unknown)  (unknown)
    @ 0xffffffffffffffff  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7f262c81f60e,7f261dfb7352,7f26ffd7f0bf,fffffffffffffffe&map=c6154ce86c34fa08dd761dfb5fd1bf97e63ed5c3:7f262c588000-7f262ce098f8,a7dce6a9f5f70e

Fold: 0
Train: [    0     1     4 ... 36762 36763 36764]
Valid: [    2     3     7 ... 36733 36736 36746]




100%|██████████| 919/919 [00:02<00:00, 408.20it/s]
https://symbolize.stripped_domain/r/?trace=56b22f,7f26ffd7f0bf&map= 
*** SIGTERM received by PID 98326 (TID 98326) on cpu 24 from PID 88996; stack trace: ***
PC: @           0x56b22f  (unknown)  _PyEval_EvalFrameDefault
    @     0x7f261dfb7353        992  (unknown)
    @     0x7f26ffd7f0c0  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=56b22f,7f261dfb7352,7f26ffd7f0bf&map=a7dce6a9f5f70e08fccb1cadb4ea57b5:7f2609b31000-7f261e3312f0 
E0607 14:11:50.059892   98326 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.
https://symbolize.stripped_domain/r/?trace=7f26ffd29edb,7f26ffd7f0bf,7f261dec96d7,7f261deca04f,7f261decb32a,7f261deca23c,7f261decac9e,7f261dfb7b2a,7f26ffd7f0bf&map=a7dce6a9f5f70e08fccb1cadb4ea57b5:7f2609b31000-7f261e3312f0 
E0607 14:11:50.073853   98326 process_state.cc:1067] RAW: Signal 11 raised at PC: 0x7f26ffd29edb while already in FailureSignalHandler!
E0607 14:11:50.073863   98326 pro

Fold: 1
Train: [    0     2     3 ... 36759 36763 36764]
Valid: [    1     4     8 ... 36760 36761 36762]




100%|██████████| 919/919 [00:02<00:00, 395.68it/s]
https://symbolize.stripped_domain/r/?trace=7f262c81f60e,7f26ffd7f0bf,fffffffffffffffe&map=c6154ce86c34fa08dd761dfb5fd1bf97e63ed5c3:7f262c588000-7f262ce098f8 
*** SIGTERM received by PID 106164 (TID 106164) on cpu 50 from PID 88996; stack trace: ***
PC: @     0x7f262c81f60e  (unknown)  _$LT$hashbrown..raw..RawTable$LT$T$C$A$GT$$u20$as$u20$core..ops..drop..Drop$GT$::drop::h8aeca1e9ba6ed7af
    @     0x7f261dfb7353        992  (unknown)
    @     0x7f26ffd7f0c0  (unknown)  (unknown)
    @ 0xffffffffffffffff  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7f262c81f60e,7f261dfb7352,7f26ffd7f0bf,fffffffffffffffe&map=c6154ce86c34fa08dd761dfb5fd1bf97e63ed5c3:7f262c588000-7f262ce098f8,a7dce6a9f5f70e08fccb1cadb4ea57b5:7f2609b31000-7f261e3312f0 
E0607 14:12:08.853503  106164 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.
https://symbolize.stripped_domain/r/?trace=7f261e013483,7f26ffd7f0bf,7f261dec5e06,7f2

Fold: 2
Train: [    0     1     2 ... 36761 36762 36763]
Valid: [    5     6     9 ... 36755 36759 36764]




100%|██████████| 229/229 [00:00<00:00, 439.17it/s]
https://symbolize.stripped_domain/r/?trace=7f26ffdd18f7,7f26ffd7f0bf&map= 
*** SIGTERM received by PID 118332 (TID 118332) on cpu 63 from PID 88996; stack trace: ***
PC: @     0x7f26ffdd18f7  (unknown)  (unknown)
    @     0x7f261dfb7353        992  (unknown)
    @     0x7f26ffd7f0c0  (unknown)  (unknown)
    @ ... and at least 1 more frames
https://symbolize.stripped_domain/r/?trace=7f26ffdd18f7,7f261dfb7352,7f26ffd7f0bf&map=a7dce6a9f5f70e08fccb1cadb4ea57b5:7f2609b31000-7f261e3312f0 
E0607 14:12:35.521093  118332 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.


Fold: 3
Train: [    0     1     2 ... 36761 36762 36764]
Valid: [   10    22    23 ... 36745 36756 36763]




100%|██████████| 229/229 [00:00<00:00, 400.49it/s]
https://symbolize.stripped_domain/r/?trace=7f26ffdd2a89,7f26ffd7f0bf&map= 
*** SIGTERM received by PID 126206 (TID 126206) on cpu 49 from PID 88996; stack trace: ***
PC: @     0x7f26ffdd2a89  (unknown)  (unknown)
    @     0x7f261dfb7353        992  (unknown)
    @     0x7f26ffd7f0c0  (unknown)  (unknown)
    @ ... and at least 1 more frames
https://symbolize.stripped_domain/r/?trace=7f26ffdd2a89,7f261dfb7352,7f26ffd7f0bf&map=a7dce6a9f5f70e08fccb1cadb4ea57b5:7f2609b31000-7f261e3312f0 
E0607 14:13:05.491091  126206 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.


Fold: 4
Train: [    1     2     3 ... 36762 36763 36764]
Valid: [    0    13    14 ... 36748 36753 36757]




In [5]:
import json
from sklearn.model_selection import StratifiedKFold

kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
## stratified Kfold for train dataframe using discourse_type and discourse_effectiveness
for fold, (train_index, valid_index) in enumerate(kf.split(train, train["discourse_type"], train["discourse_effectiveness"])):

    train_temp = train.iloc[train_index]
    valid_temp = train.iloc[valid_index]

    train_data = prepare_training_data(train_temp, tokenizer, cfg, num_jobs=32, is_train=True)
    val_data = prepare_training_data(valid_temp, tokenizer, cfg, num_jobs=32, is_train=True)

    df = pd.DataFrame.from_records(train_data)
    df.to_json(f"/kaggle/working/folds/train_{fold}.jsonl", orient="records", lines=True)

    df = pd.DataFrame.from_records(val_data)
    df.to_json(f"/kaggle/working/folds/valid_{fold}.jsonl", orient="records", lines=True)

    print("Fold:", fold)
    print("Train:", train_index)
    print("Valid:", valid_index)
    print("\n")

100%|██████████| 229/229 [00:00<00:00, 437.84it/s]
https://symbolize.stripped_domain/r/?trace=7f262c81f63f,7f26ffd7f0bf,fffffffffffffffe&map=c6154ce86c34fa08dd761dfb5fd1bf97e63ed5c3:7f262c588000-7f262ce098f8 
*** SIGTERM received by PID 134161 (TID 134161) on cpu 2 from PID 88996; stack trace: ***
https://symbolize.stripped_domain/r/?trace=7f26ffdd1a00,7f26ffd7f0bf&map= 
*** SIGTERM received by PID 134162 (TID 134162) on cpu 6 from PID 88996; stack trace: ***
PC: @     0x7f26ffdd1a00  (unknown)  (unknown)
    @     0x7f261dfb7353        992  (unknown)
    @     0x7f26ffd7f0c0  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7f26ffdd1a00,7f261dfb7352,7f26ffd7f0bf&map=a7dce6a9f5f70e08fccb1cadb4ea57b5:7f2609b31000-7f261e3312f0 
E0607 14:14:24.317731  134162 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.
PC: @     0x7f262c81f63f  (unknown)  _$LT$hashbrown..raw..RawTable$LT$T$C$A$GT$$u20$as$u20$core..ops..drop..Drop$GT$::drop::h8aeca1e9ba6ed7af
    @

Fold: 0
Train: [    0     1     4 ... 36762 36763 36764]
Valid: [    2     3     7 ... 36733 36736 36746]




100%|██████████| 229/229 [00:00<00:00, 420.29it/s]
https://symbolize.stripped_domain/r/?trace=7f26ffdd1a06,7f26ffd7f0bf&map= 
*** SIGTERM received by PID 142055 (TID 142055) on cpu 17 from PID 88996; stack trace: ***
PC: @     0x7f26ffdd1a06  (unknown)  (unknown)
    @     0x7f261dfb7353        992  (unknown)
    @     0x7f26ffd7f0c0  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7f26ffdd1a06,7f261dfb7352,7f26ffd7f0bf&map=a7dce6a9f5f70e08fccb1cadb4ea57b5:7f2609b31000-7f261e3312f0 
E0607 14:14:53.473831  142055 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.


Fold: 1
Train: [    0     2     3 ... 36759 36763 36764]
Valid: [    1     4     8 ... 36760 36761 36762]




100%|██████████| 229/229 [00:00<00:00, 451.62it/s]
https://symbolize.stripped_domain/r/?trace=7f262c81f63f,7f26ffd7f0bf,fffffffffffffffe&map=c6154ce86c34fa08dd761dfb5fd1bf97e63ed5c3:7f262c588000-7f262ce098f8 
*** SIGTERM received by PID 149921 (TID 149921) on cpu 72 from PID 88996; stack trace: ***
PC: @     0x7f262c81f63f  (unknown)  _$LT$hashbrown..raw..RawTable$LT$T$C$A$GT$$u20$as$u20$core..ops..drop..Drop$GT$::drop::h8aeca1e9ba6ed7af
    @     0x7f261dfb7353        992  (unknown)
    @     0x7f26ffd7f0c0  (unknown)  (unknown)
    @ 0xffffffffffffffff  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7f262c81f63f,7f261dfb7352,7f26ffd7f0bf,fffffffffffffffe&map=c6154ce86c34fa08dd761dfb5fd1bf97e63ed5c3:7f262c588000-7f262ce098f8,a7dce6a9f5f70e08fccb1cadb4ea57b5:7f2609b31000-7f261e3312f0 
E0607 14:15:22.905269  149921 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.
https://symbolize.stripped_domain/r/?trace=7f261e013483,7f26ffd7f0bf,7f261dec5e06,7f2

Fold: 2
Train: [    0     1     2 ... 36761 36762 36763]
Valid: [    5     6     9 ... 36755 36759 36764]




100%|██████████| 229/229 [00:00<00:00, 442.38it/s]
https://symbolize.stripped_domain/r/?trace=7f262c81f63f,7f26ffd7f0bf,fffffffffffffffe&map=c6154ce86c34fa08dd761dfb5fd1bf97e63ed5c3:7f262c588000-7f262ce098f8 
*** SIGTERM received by PID 157814 (TID 157814) on cpu 38 from PID 88996; stack trace: ***
PC: @     0x7f262c81f63f  (unknown)  _$LT$hashbrown..raw..RawTable$LT$T$C$A$GT$$u20$as$u20$core..ops..drop..Drop$GT$::drop::h8aeca1e9ba6ed7af
    @     0x7f261dfb7353        992  (unknown)
    @     0x7f26ffd7f0c0  (unknown)  (unknown)
    @ 0xffffffffffffffff  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=7f262c81f63f,7f261dfb7352,7f26ffd7f0bf,fffffffffffffffe&map=c6154ce86c34fa08dd761dfb5fd1bf97e63ed5c3:7f262c588000-7f262ce098f8,a7dce6a9f5f70e08fccb1cadb4ea57b5:7f2609b31000-7f261e3312f0 
E0607 14:15:42.491092  157814 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.
https://symbolize.stripped_domain/r/?trace=7f26ffd29edb,7f26ffd7f0bf,7f261dec96d7,7f2

Fold: 3
Train: [    0     1     2 ... 36761 36762 36764]
Valid: [   10    22    23 ... 36745 36756 36763]




100%|██████████| 919/919 [00:02<00:00, 375.02it/s]
https://symbolize.stripped_domain/r/?trace=56ab67,7f26ffd7f0bf&map= 
*** SIGTERM received by PID 161375 (TID 161375) on cpu 62 from PID 88996; stack trace: ***
PC: @           0x56ab67  (unknown)  _PyEval_EvalFrameDefault
    @     0x7f261dfb7353        992  (unknown)
    @     0x7f26ffd7f0c0  (unknown)  (unknown)
https://symbolize.stripped_domain/r/?trace=56ab67,7f261dfb7352,7f26ffd7f0bf&map=a7dce6a9f5f70e08fccb1cadb4ea57b5:7f2609b31000-7f261e3312f0 
E0607 14:15:54.944597  161375 coredump_hook.cc:320] RAW: Remote crash gathering disabled for SIGTERM.
https://symbolize.stripped_domain/r/?trace=7f26ffd29edb,7f26ffd7f0bf,7f261dec96d7,7f261deca04f,7f261decb32a,7f261deca23c,7f261decac9e,7f261dfb7b2a,7f26ffd7f0bf&map=a7dce6a9f5f70e08fccb1cadb4ea57b5:7f2609b31000-7f261e3312f0 
E0607 14:15:54.958459  161375 process_state.cc:1067] RAW: Signal 11 raised at PC: 0x7f26ffd29edb while already in FailureSignalHandler!
E0607 14:15:54.958470  161375 p

Fold: 4
Train: [    1     2     3 ... 36762 36763 36764]
Valid: [    0    13    14 ... 36748 36753 36757]




In [8]:
dataset = load_dataset("json", data_files="/kaggle/working/folds/train_0.jsonl", split="train")

Extracting data files: 100%|██████████| 1/1 [00:00<00:00, 805.67it/s]
                                

Dataset json downloaded and prepared to /root/.cache/huggingface/datasets/json/default-d6884a81e6d35181/0.0.0/da492aad5680612e4028e7f6ddc04b1dfcec4b64db470ed7cc5f2bb265b9b6b5. Subsequent calls will reuse this data.


In [10]:
tokenizer.pad_token_id

0